## EMA Cross with vectorBT

最近沒精神，水一篇，最基本的 EMA 金叉死叉，loop 看那個短週期穿長週期收益較高，vectorBT 的運行效率似乎比 backtesting 高，可以試試

In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
df = yf.download("^SPX", period="60d", interval="5m")

df.index = df.index.tz_convert('Asia/Shanghai')

[*********************100%%**********************]  1 of 1 completed


In [3]:
def calc_return(short_span, long_span):
    short_ema = vbt.MA.run(data, short_span, short_name='fast', ewm=True)
    long_ema = vbt.MA.run(data, long_span, short_name='slow', ewm=True)

    # Generate crossover signals
    entries = short_ema.ma_crossed_above(long_ema)
    exits = short_ema.ma_crossed_below(long_ema)
    portfolio = vbt.Portfolio.from_signals(
        data, entries, exits,
        size=0.1, size_type='amount', fees=0.0001,
        init_cash=100000, freq='5m',
        # min_size =1, size_granularity = 1
    )
    return portfolio

In [4]:
data = df.Close

# Parameter grid
short_spans = np.arange(5, 15, 1)  # Test spans from 5 to 14
long_spans = np.arange(15, 60, 1)  # Test spans from 15 to 59

# Backtesting with different EMA spans
results = []
for short_span in short_spans:
    for long_span in long_spans:
        portfolio = calc_return(short_span, long_span)
        results.append((short_span, long_span, portfolio.total_return()))

# Find the best performing parameter set
best_params = max(results, key=lambda x: x[2])
print("Best Parameters:", best_params)

Best Parameters: (10, 55, 0.00044860165751953786)


In [5]:
short, long, total_return = best_params

pf = calc_return(short, long)

In [6]:
pf.value().vbt.plot()

FigureWidget({
    'data': [{'showlegend': False,
              'type': 'scatter',
              'uid': '274568a8-d526-4295-9a57-41ff317e6ede',
              'x': array([datetime.datetime(2023, 12, 4, 22, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
                          datetime.datetime(2023, 12, 4, 22, 35, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
                          datetime.datetime(2023, 12, 4, 22, 40, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
                          ...,
                          datetime.datetime(2024, 2, 28, 4, 45, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
                          datetime.datetime(2024, 2, 28, 4, 50, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
                          datetime.datetime(2024, 2, 28, 4, 55, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>)],
                         dtype=object),
              'y': array([100000.        , 100000.        , 100000.        , ..

In [7]:
pf.total_profit() # 總收益

44.86016575195379

In [8]:
pf.stats()

Start                         2023-12-04 22:30:00+08:00
End                           2024-02-28 04:55:00+08:00
Period                                 15 days 17:00:00
Start Value                                    100000.0
End Value                                 100044.860166
Total Return [%]                                0.04486
Benchmark Return [%]                          11.231598
Max Gross Exposure [%]                         0.510799
Total Fees Paid                                4.475528
Max Drawdown [%]                               0.009666
Max Drawdown Duration                   4 days 20:35:00
Total Trades                                         47
Total Closed Trades                                  46
Total Open Trades                                     1
Open Trade PnL                                 0.656258
Win Rate [%]                                  36.956522
Best Trade [%]                                 2.322553
Worst Trade [%]                                -

In [9]:
pf.trades.stats()

Start                         2023-12-04 22:30:00+08:00
End                           2024-02-28 04:55:00+08:00
Period                                 15 days 17:00:00
First Trade Start             2023-12-05 03:45:00+08:00
Last Trade End                2024-02-28 04:55:00+08:00
Coverage                                9 days 22:25:00
Overlap Coverage                        0 days 00:00:00
Total Records                                        47
Total Long Trades                                    47
Total Short Trades                                    0
Total Closed Trades                                  46
Total Open Trades                                     1
Open Trade PnL                                 0.656258
Win Rate [%]                                  36.956522
Max Win Streak                                        3
Max Loss Streak                                       8
Best Trade [%]                                 2.322553
Worst Trade [%]                                -

In [10]:
pf.trades.records_readable

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,0,0.1,2023-12-05 03:45:00+08:00,4563.850098,0.045639,2023-12-05 04:05:00+08:00,4560.959961,0.045610,-0.380262,-0.000833,Long,Closed,0
1,1,0,0.1,2023-12-05 04:30:00+08:00,4565.520020,0.045655,2023-12-05 22:35:00+08:00,4552.919922,0.045529,-1.351194,-0.002960,Long,Closed,1
2,2,0,0.1,2023-12-05 23:05:00+08:00,4566.669922,0.045667,2023-12-06 01:15:00+08:00,4564.689941,0.045647,-0.289312,-0.000634,Long,Closed,2
3,3,0,0.1,2023-12-06 02:25:00+08:00,4565.930176,0.045659,2023-12-07 00:35:00+08:00,4565.589844,0.045656,-0.125348,-0.000275,Long,Closed,3
4,4,0,0.1,2023-12-07 22:50:00+08:00,4568.970215,0.045690,2023-12-09 00:45:00+08:00,4583.799805,0.045838,1.391431,0.003045,Long,Closed,4
5,5,0,0.1,2023-12-09 01:40:00+08:00,4591.149902,0.045911,2023-12-14 02:20:00+08:00,4644.310059,0.046443,5.223661,0.011378,Long,Closed,5
6,6,0,0.1,2023-12-14 02:55:00+08:00,4646.160156,0.046462,2023-12-15 02:40:00+08:00,4708.669922,0.047087,6.157428,0.013253,Long,Closed,6
7,7,0,0.1,2023-12-15 03:55:00+08:00,4716.359863,0.047164,2023-12-16 02:00:00+08:00,4714.279785,0.047143,-0.302314,-0.000641,Long,Closed,7
8,8,0,0.1,2023-12-16 04:50:00+08:00,4724.359863,0.047244,2023-12-21 03:10:00+08:00,4769.169922,0.047692,4.386071,0.009284,Long,Closed,8
9,9,0,0.1,2023-12-22 04:15:00+08:00,4729.160156,0.047292,2023-12-23 03:15:00+08:00,4749.100098,0.047491,1.899212,0.004016,Long,Closed,9
